In [6]:
import json
import copy
import random
import time

#Functions for generating game states
def generate_next_gameloop(current_state):
    """
    Generates the next game loop in a game based on the current state.

    :param current_state: A JSON object representing the current state of the game.
    :return: A JSON object representing the next state of the game.
    """
    # Deep copy the current state to avoid modifying the original state
    new_state = copy.deepcopy(current_state)

    # Increment the game loop number
    new_state['gameloop']['loopNo'] += 1

    # Process each bike and its agents
    for bike in new_state['gameloop']['bikes'].values():
        # Check if the Audi intersects this bike
        audi_intersects = new_state['gameloop']['audi']['target'] == bike['id']

        # Process each agent on the bike
        for agent in bike['agents'].values():
            # If the Audi intersects, remove all agents from the bike
            if audi_intersects:
                agent['energy'] = 0  # Setting energy to 0 as they are eliminated

            # Update agent's energy and check if they should be moved to bike 0
            if agent['energy'] > 0:
                agent['energy'] -= 1  # Decrease energy by 1 for this loop
                if agent['energy'] == 0:
                    # Move to bike 0
                    agent['bikeID'] = '0x0'
            else:
                # Already on bike 0 or moved to bike 0
                agent['bikeID'] = '0x0'

    # Handle agents on bike 0 (this can be expanded with more logic if needed)
    # Currently, it only ensures they are on bike 0

    return new_state

def generate_initial_state():
    """
    Generates an initial game state with a random number of bikes and agents.

    :return: A JSON object representing the initial state of the game.
    """
    # Define possible colors for agents
    possible_colours = ["Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "White"]

    # Initialize the game state
    initial_state = {
        "gameloop": {
            "loopNo": 0,
            "bikes": {},
            "lootboxes": {},  # This can be populated with lootbox data if needed
            "audi": {
                "target": None,  # No target initially
                "position": "Unknown",
            }
        },
    }

    # Randomly determine the number of bikes
    num_bikes = random.randint(6, 10)

    for bike_id in range(1, num_bikes + 1):
        # Each bike will have a unique ID
        bike_key = f"bike_{bike_id}"

        # Randomly determine the number of agents on this bike
        num_agents = random.randint(6, 10)
        bike_agents = {}

        for agent_id in range(1, num_agents + 1):
            # Each agent will have a unique ID and randomly assigned color
            agent_key = f"agent_{bike_id}_{agent_id}"
            agent_colour = random.choice(possible_colours)

            # Define the agent
            bike_agents[agent_key] = {
                "id": agent_key,
                "groupID": f"group_{bike_id}",
                "bikeID": bike_key,
                "colour": agent_colour,
                "position": "Unknown",
                "points": 0,
                "energy": 100,  # Initial energy can be adjusted
                # The rest of the agent's attributes are placeholders
                "bikes": {},
                "forces": {},
                "lootboxChoice": None,
                "lootDistribution": [],
            }

        # Add the bike and its agents to the game state
        initial_state["gameloop"]["bikes"][bike_key] = {
            "id": bike_key,
            "agents": bike_agents,
        }

    return initial_state


In [7]:
# Generate the initial state and run the game for LOOPS number of loops
initial_state = generate_initial_state()
LOOPS = 10
for i in range(LOOPS):
    initial_state = generate_next_gameloop(initial_state)
with open(f"./visualiser/JSON/{time.time()}.json", "w") as f:
    json.dump(initial_state, f, indent=4)

In [8]:
import pygame
import pygame_gui
import visualiser.Constants as CONST
from visualiser.entities import Agents, Lootboxes, Awdi, Bikes
import pygame.locals as PYG

pygame.init()
window = pygame.display.set_mode((CONST.SCREEN_WIDTH, CONST.SCREEN_HEIGHT))
pygame.display.set_caption("Visualiser")
manager = pygame_gui.UIManager((CONST.SCREEN_WIDTH, CONST.SCREEN_HEIGHT))
clock = pygame.time.Clock()
hello_button = pygame_gui.elements.UIButton(relative_rect=pygame.Rect((350, 275), (100, 50)),
                                             text='Say Hello',
                                             manager=manager)
running = True

while running:
    time_delta = clock.tick(60) / 1000.0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == PYG.KEYDOWN:
            if event.key == PYG.K_ESCAPE:
                running = False
        if event.type == pygame_gui.UI_BUTTON_PRESSED:
            if event.ui_element == hello_button:
                print('Hello World!')
        manager.process_events(event)
    
    manager.update(time_delta)
    window.fill((0, 0, 0))
    pygame.draw.circle(window, (255, 255, 255), (250, 250), 250, 1)
    manager.draw_ui(window)
    pygame.display.flip()
pygame.quit()


In [1]:
%load_ext autoreload
%autoreload 2
from Visualiser import Visualiser
a = Visualiser()
a.run_loop()

pygame-ce 2.3.2 (SDL 2.26.5, Python 3.10.9)


RuntimeError: The FreeType 2 library hasn't been initialized

: 